In [133]:
#Seq2Seq (Sequence-to-Sequence) is a type of model in machine learning that is used for tasks such as machine translation, text summarization, and image captioning. 
#The model consists of two main components:Encoder &  Decoder
#Seq2Seq models are trained using a dataset of input-output pairs, where the input is a sequence of tokens and the output is also a sequence of tokens.
#The model is trained to maximize the likelihood of the correct output sequence given the input sequence.
#Encoder:It uses deep neural network layers and converts the input words to corresponding hidden vectors.
#Each vector represents the current word and the context of the word
#decoder:It takes as input the hidden vector generated by the encoder, its own hidden states, and the current word to produce the next hidden vector and finally predict the next word.

In [134]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [135]:
# Define parameters (should be consistent with the trained model)
vocab_size = 50 #size of the vocabulary (number of unique words)
embedding_dim = 10 # specifies the dimensionality of the embedding vectors.
hidden_units = 16 #no.of nodes in LSTM
sequence_length = 10 #no.of squence
num_encoder_layers = 4
num_decoder_layers = 4
hidden2 = 16

In [136]:
# Define the model architecture (same as during training)
#Encoder:It uses deep neural network layers and converts the input words to corresponding hidden vectors.
encoder_inputs = Input(shape=(sequence_length,)) #input layer for the encode
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs) #defines an embedding layer that converts the input sequences into dense vectors.
#encoder_lstm = LSTM(hidden_units, return_state=True)
#encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding) #state_h and state_c, which are the hidden state and cell state at the final time step.
for _ in range(num_encoder_layers):
    #encoder_outputs, state_h, state_c = LSTM(hidden2, return_state=True)(encoder_outputs)
    encoder_lstm = LSTM(hidden_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding) #state_h and state_c, which are the hidden state and cell state at the final time step.

encoder_states = [state_h, state_c]

In [137]:
#decoder:It takes as input the hidden vector generated by the encoder, its own hidden states, and the current word to produce the next hidden vector and finally predict the next word.
decoder_inputs = Input(shape=(sequence_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
for _ in range(num_encoder_layers):
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

In [138]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [139]:
#cell state allowing information to flow through the sequence 
#The hidden state in an LSTM represents the network's memory of past information.
#the hidden state and cell state work together to enable LSTMs to process and remember sequential information.
import numpy as np
def predict_sequence(input_sequence):
    input_sequence = np.array(input_sequence)
    encoder_model = Model(encoder_inputs, encoder_states)

    # Initialize the decoder's initial states with the encoder's final states
    decoder_state_input_h = Input(shape=(hidden_units,))
    decoder_state_input_c = Input(shape=(hidden_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding, initial_state=decoder_states_inputs
    )
    decoder_states = [state_h, state_c]        
    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states
    )
    
# Start the prediction process
    states_values = encoder_model.predict(input_sequence)
    target_seq = np.zeros((1, 1)) #The shape (1, 1) indicates that it's a sequence of length 1 and the token index is set to 0.
    target_seq[0, 0] = 1  #The code assigns a specific value (1 in this case) to the first token in the target sequence

    decoded_sequence = []

    for _ in range(sequence_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_values) # The decoder model is used to predict the next token in the sequence. 

        sampled_token_index = np.argmax(output_tokens[0, -1, :]) #The index of the token with the highest predicted probability is chosen as the next token in the sequence. 
        decoded_sequence.append(sampled_token_index)

        target_seq[0, 0] = sampled_token_index #The target_seq is updated to hold the last predicted token index.
        states_values = [h, c]

    return decoded_sequence

In [140]:
input_sequence = [1, 2, 3, 4, 5, 6, 7,8,9,10]  # Example input sequence
predicted_sequence = predict_sequence([input_sequence])
print("Predicted Sequence:", predicted_sequence)

1/1 [==============================] - 0s 24ms/step
Predicted Sequence: [28, 28, 26, 12, 39, 12, 35, 36, 36, 32]
